In [89]:
from copy import deepcopy
import datetime
import json
import pandas as pd

In [157]:
df = pd.read_csv('boston_listing_dates.csv')

In [160]:
df.shape

(417098, 9)

In [161]:
df.head()

,property_id,zipcode,list_date,sale_date,withdrawn_date,expired_date,cancelled_date,current_status,sale_price
0,68560382,1880,2016-03-01,2016-04-13,NaN,NaN,NaN,Sale,485000.0
1,504929421,1440,2016-03-01,NaN,NaN,NaN,2016-05-02,Cancelled,NaN
2,347490903,2130,2016-03-01,2016-04-22,NaN,NaN,NaN,Sale,772000.0
3,66726260,2720,2016-03-01,2016-05-18,NaN,NaN,NaN,Sale,137194.0
4,645765969,2301,2016-03-01,2016-07-08,NaN,NaN,NaN,Sale,405000.0


In [149]:
# for column in 'list_date sale_date withdrawn_date expired_date cancelled_date'.split():
#     df[column] = pd.to_datetime(df[column]).dt.date

In [162]:
def convert_to_zipcode(zip_int):
    zipcode = str(zip_int)
    if len(zipcode) == 4:
        return '0' + zipcode
    return zipcode
df['zipcode'] = df['zipcode'].apply(convert_to_zipcode)

In [151]:
# def convert_date_to_idx(date):
#     return (date - start_date).days
# def convert_idx_to_date(idx):
#     return start_date + datetime.timedelta(days=idx)

In [169]:
start_date = df.list_date[0]
date_today = datetime.date.today()
date_today_str = str(date_today)
dates = [str(date.date()) for date in pd.date_range(start_date, date_today, freq='d')]
zipcodes = set(df.zipcode)

In [164]:
category_to_cnt = {cat: 0 for cat in ['new_listings', 'sales']}
date_to_category_to_cnt = {date: deepcopy(category_to_cnt) for date in dates}
zipcode_to_date_to_category_to_cnt = {zipcode: deepcopy(date_to_category_to_cnt)
                                          for zipcode in zipcodes}

In [167]:
date_to_category_to_market_area_cnt = deepcopy(date_to_category_to_cnt)

In [173]:
for idx, row in df.iterrows():
    zipcode = row['zipcode']
    list_date = row['list_date']
    sale_date = row['sale_date']
    if list_date <= date_today_str:
        zipcode_to_date_to_category_to_cnt[zipcode][list_date]['new_listings'] += 1
        date_to_category_to_market_area_cnt[list_date]['new_listings'] += 1
    if pd.notnull(sale_date) and sale_date <= date_today_str:
        zipcode_to_date_to_category_to_cnt[zipcode][sale_date]['sales'] += 1
        date_to_category_to_market_area_cnt[sale_date]['sales'] += 1

In [176]:
with open('zipcode_to_date_to_category_to_cnt.txt', 'w') as f:
    f.write(json.dumps(zipcode_to_date_to_category_to_cnt))
with open('date_to_category_to_market_area_cnt.txt', 'w') as f:
    f.write(json.dumps(date_to_category_to_market_area_cnt))